### ITCR - Escuela de Computación
### Curso IC-6200 - Inteligencia Artificial
### Aprendizaje supervisado

### Redes de memoria de corto y largo plazo con PyTorch 
### (Long-Short Term Memory Networks-LSTM)

**Profesora: María Auxiliadora Mora**

### Tarea corta #7

### Estudiantes:
**1. Pablo Alberto Muñoz Hidalgo**

**2. Luis Andrés Rojas Murillo**


## Introducción

La clasificación de textos y el reconocimiento de entidades nombradas (Named Entity Recognition o NER por sus siglas en inglés) son técnicas fundamentales que constituyen el primer paso en muchas tareas de Procesamiento de lenguaje natural (NLP). NER, es un área de investigación relacionada a la extracción de información, que permite localizar y clasificar nombres de entidades que se encuentran en texto libre, en categorías comunmente organizaciones, lugares, tiempo, personas, entre otros. Ejemplo:

- El fundador de [Microsoft Corporation] (organización), [Bill Gates] (persona), comentó que se abrirán 1000 puestos de trabajo en la [Región Chorotega] (lugar) a partir del año 2022 (fecha).  

La clasificación de textos permite categorizar el contenido asociando este a un conjunto de etiquetas predefinidas o clases. Su uso más popular es el análisis de sentimientos. Ejemplo:

- En mi opinión, la película fue muy buena porque pudo dar a conocer a los espectadores cómo puede afectar una situación traumática a la mente humana. (Clase = 5 o excelente). 

Las redes neuronales recurrentes o RNN (Rumelhart et al., 1986, como se citó en LeCun et al., 2015) son una familia de redes neuronales para el procesamiento de secuencias de datos, las cuales en un tiempo t, reciben el estado anterior, es decir, su salida en el tiempo t podría usarse como insumo del procesamiento de la siguiente entrada, de modo que la información pueda propagarse a medida que la red pasa por la secuencia de entrada. Las redes Long Short-Term Memory (LSTM) son un tipo de red neuronal recurrente capaz de aprender dependencias a largo plazo.

El siguiente ejemplo implementa NER con una LSTM para etiquetar el rol que juegan las palabras en las oraciones. 


## Ejemplo

El sistema implementado en el código adjunto soluciona el problema de estimar el rol de una palabra en una frase, por ejempo roles como determinante (DET), nombre (NN) y verbo (V). 
Ejemplo para la frase:

- "El perro come manzana" la salida deberá ser: ["DET", "NN", "V", "NN"]). 

Este proceso se conoce en el procesamiento de lenguaje natural como "part of speech tagging (POS)".

Este es un ejemplo simple con datos introducidos en el código basado en [1].

Se realizarán los siguientes pasos

   * Definición de los ejemplos (codificados) 
   * Preprocesamiento de las palabras a clasificar
   * Definición del modelo
   * Instanciación del modelo, definición de la función de pérdida y del optimizador  
   * Entrenamiento de la red
   * Pruebas del modelo resultante con unos cuantos ejemplos.


In [62]:
# Bibliotecas requeridas

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import random

torch.manual_seed(1)

In [63]:
# Funciones utilitarias

def max_values(x):
    """
    Retorna el valor máximo y en índice o la posición del valor en un vector x.
    Parámetros: 
        x: vector con los datos. 
    Salida: 
        out: valor 
        inds: índice
    """
    out, inds = torch.max(x,dim=1)   
    return out, inds
    

    # Preparación de los datos 
def prepare_sequence(seq, to_ix):
    """
    Retorna un tensor con los indices del diccionario para cada palabras en una oración.
    Parámetros:
       seq: oración
       to_ix: diccionario de palabras.
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


In [66]:
#limpieza de datos 
# Read the data from the CSV file
labels = ["DET", "NN", "V", "ADJ", "PREP"]
with open('FULL DATA.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

with open('FULL DATA.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)
counter = 0
list_to_delete = []

#todas las fraces con otra dificultad
for i in data:
    if len(i) > 8 or len(i) < 8:
        counter += 1
        list_to_delete.append(i)

for i in list_to_delete:
    data.remove(i)
list_to_delete = []

#todas las fraces con diferete logitud
for i in data:
    sentence = i[0].split()
    if len(sentence) > 7 or len(sentence) < 7:
        counter += 1
        list_to_delete.append(i)

for i in list_to_delete:
    data.remove(i)
list_to_delete = []

#todas las fraces con etiquetas no reconocidas
for i in data:
    for j in range(len(i)):
        if j > 0 and i[j] not in labels:
            counter += 1
            list_to_delete.append(i)


for i in list_to_delete:
    if i in data:
        data.remove(i)
list_to_delete = []

print("Numero de frases con errores: ", counter)
print("Numero de frases sin errores: ", len(data))


ValueError: list.remove(x): x not in list

In [ ]:


# Divide the data into training and testing sets
#random.shuffle(data)
split = int(0.8 * len(data))
training_data = data[:split]
test_data = data[split:]


# Split the training data into text and labels
for i in range(len(training_data)):
    splitted_text = training_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(training_data[i])):
        if j != 0:
            labels.append(training_data[i][j])
    training_data[i] = [0,0]
    training_data[i][0] = splitted_text
    training_data[i][1] = labels

# Split the test data into text and labels
for i in range(len(test_data)):
    splitted_text = test_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(test_data[i])):
        if j != 0:
            labels.append(test_data[i][j])
    test_data[i] = [0,0]
    test_data[i][0] = splitted_text
    test_data[i][1] = labels

# print the data
print("Training data:")
print(training_data)
print("Test data:")
print(test_data)


# Diccionario las palabras
word_to_ix = {}
for sent, tags in training_data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print("Diccionario", word_to_ix)

# Asignar índices a las etiquetas
tag_to_ix = {"DET": 0, "NN": 1, "V": 2, "ADJ":3, "PREP":4}

#print the amount of each tag found in the text, print that in a table using pandas

print("\n\n\n\nCantidad de etiquetas encontradas en el texto:")
for tag in tag_to_ix:
    count = 0
    for sent, tags in training_data + test_data:
        for word in tags:
            if word == tag:
                count += 1
    print(tag," : ", count)


for i in range(len(training_data)):
    #put the rest of i in other list
    if len(training_data[i][1]) == 6:
        print(training_data[i])

for i in range(len(test_data)):
    #put the rest of i in other list
    if len(test_data[i][1]) == 6:
        print(test_data[i])



Training data:
[[['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardÃ\xadn'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'mesa', 'grande', 'estÃ¡', 'en', 'la', 'cocina'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'perro', 'marrÃ³n', 'ladra', 'en', 'el', 'parque'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'libro', 'interesante', 'estÃ¡', 'en', 'la', 'biblioteca'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'niÃ±a', 'pequeÃ±a', 'juega', 'en', 'el', 'patio'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'pelota', 'roja', 'rueda', 'en', 'el', 'suelo'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'hombre', 'mayor', 'camina', 'por', 'la', 'calle'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'mujer', 'guapa', 'estÃ¡', 'en', 'el', 'parque'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'niÃ±o', 'feliz', 'juega', 'en', 'la', 'playa'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['E

In [ ]:
# Ejemplo de procesamiento de una oración
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])                          
print(inputs)

['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardÃ\xadn']
tensor([0, 1, 2, 3, 4, 5, 6])


In [ ]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    """
    Clase para aplicar POST a oraciones en español. 
    """
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        """
        Inicialización de la clase.
        Parámetros:
           embedding_dim: dimesionalidad del vector de palabras. 
           hidden_dim: dimensión de la capa oculta de la red. 
           vocab_size: tamaño del vocabulario.  
           tagset_size: número de clases.
        """
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. Ejemplos en [3] y [4]
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera los estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de clases
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pasada hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver la probabilidad de cada etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    

#============================================================
#BORRAR AL FINAL DESDE AQUI O MOVERLAS A CELDAS POR SEPARADO  
#============================================================

# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
EMBEDDING_DIM = 6
# Valores generalmente altos (32 o 64 dimensiones).
HIDDEN_DIM = 6


# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con múltiples clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)

#============================================================
#BORRAR AL FINAL DESDE AQUI O MOVERLAS A CELDAS POR SEPARADO
#============================================================


# Entrenamiento del modelo 

# Valores antes de entrenar
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Épocas de entrenamiento
for epoch in range(200):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)


['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardÃ\xadn']
tensor([[-2.0294, -1.4995, -1.4343, -1.7211, -1.4776],
        [-2.1721, -1.5072, -1.4454, -1.7485, -1.3671],
        [-2.1510, -1.3367, -1.5995, -1.6185, -1.5107],
        [-2.1193, -1.3589, -1.5976, -1.6407, -1.4842],
        [-2.0679, -1.4440, -1.6056, -1.6738, -1.3892],
        [-2.1571, -1.3371, -1.6179, -1.6046, -1.5030],
        [-2.0911, -1.4176, -1.5764, -1.6657, -1.4338]])


ValueError: Expected input batch_size (8) to match target batch_size (7).

In [ ]:
# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con múltiples clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [ ]:
# Entrenamiento del modelo 

# Valores antes de entrenar
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Épocas de entrenamiento
for epoch in range(200):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

['el', 'coche', 'rojo', 'esta', 'en', 'la', 'calle']
tensor([[-1.7479, -1.5793, -1.5727, -1.4541, -1.7222],
        [-1.7488, -1.5726, -1.5877, -1.4649, -1.6982],
        [-1.7441, -1.4946, -1.5705, -1.4701, -1.8140],
        [-1.7620, -1.5551, -1.5399, -1.4576, -1.7724],
        [-1.7356, -1.5184, -1.5611, -1.5095, -1.7503],
        [-1.7388, -1.6100, -1.5663, -1.5102, -1.6363],
        [-1.7565, -1.5474, -1.5551, -1.4710, -1.7504]])
Resultados luego del entrenamiento para la primera frase
tensor([[-3.6377e-03, -6.2140e+00, -9.4082e+00, -1.1041e+01, -6.4813e+00],
        [-7.1440e+00, -4.1712e-03, -9.8455e+00, -5.7280e+00, -9.6226e+00],
        [-1.1161e+01, -5.9205e+00, -4.2425e+00, -1.7274e-02, -9.7956e+00],
        [-8.8833e+00, -9.1328e+00, -1.1745e-02, -5.0667e+00, -5.2734e+00],
        [-5.9632e+00, -6.8017e+00, -5.1643e+00, -9.7382e+00, -9.5044e-03],
        [-5.7834e-03, -6.0287e+00, -1.1228e+01, -1.3142e+01, -5.7010e+00],
        [-8.2321e+00, -5.3538e-03, -8.8830e+00, -5.339

In [ ]:
# Uso del modelo generado

def test_examples(test_data):

   with torch.no_grad():
      inputs = prepare_sequence(test_data, word_to_ix)
      tag_scores = model(inputs)
    
 
   print("FRASE") 
   print("La frase original", test_data)    
   print("La frase original preprocesada", inputs)
   print("Salida del modelo", tag_scores)
   print("Valores máximos e índices", max_values(tag_scores))    
    
print("Clases")
print(tag_to_ix)

#Frase 1
# Las palabras en una oración se pueden etiquetar de tres formas.
# La primera oración tiene 3 palabras "El perro juega"
# por eso el tensor de salida tiene 3 elementos. 
# Cada elemento es un vector de probabilidad de estar asociada a una clase. 
# Es decir hay que calcular la posición del valor máximo. 
#   Ejemplo 1: "El perro juega" ["DET", "NN", "V"]
# Ejemplo: salida 0, 1, 2 con {"DET": 0, "NN": 1, "V": 2} => DET, NN, V 
test_examples(test_data[0][0])


Clases
{'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}
FRASE
La frase original ['el', 'viento', 'fresco', 'sopla', 'entre', 'los', 'arboles']
La frase original preprocesada tensor([ 0, 61, 62, 63, 64, 59, 65])
Salida del modelo tensor([[-3.6377e-03, -6.2140e+00, -9.4082e+00, -1.1041e+01, -6.4813e+00],
        [-7.0259e+00, -7.0815e-03, -9.1450e+00, -5.1159e+00, -9.7112e+00],
        [-4.2740e+00, -2.2147e+00, -2.7350e+00, -2.1087e-01, -6.1577e+00],
        [-7.2489e+00, -7.7990e+00, -2.2352e-02, -4.3518e+00, -4.8159e+00],
        [-2.3588e+00, -4.6996e+00, -2.0706e+00, -5.2541e+00, -2.6784e-01],
        [-2.8370e-02, -4.0488e+00, -9.3408e+00, -1.0269e+01, -4.5654e+00],
        [-5.3606e+00, -7.3568e-03, -9.9458e+00, -6.0183e+00, -8.8042e+00]])
Valores máximos e índices (tensor([-0.0036, -0.0071, -0.2109, -0.0224, -0.2678, -0.0284, -0.0074]), tensor([0, 1, 3, 2, 4, 0, 1]))


In [ ]:
#Frase 2
test_examples(test_data[3][0])

print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['el', 'sol', 'radiante', 'brilla', 'en', 'el', 'horizonte']
La frase original preprocesada tensor([ 0, 69, 70, 71,  4,  0, 72])
Salida del modelo tensor([[-3.6377e-03, -6.2140e+00, -9.4082e+00, -1.1041e+01, -6.4813e+00],
        [-4.6229e+00, -1.1412e-02, -1.0982e+01, -6.5131e+00, -1.0709e+01],
        [-5.1358e+00, -1.4691e+00, -4.6898e+00, -2.8190e-01, -7.7391e+00],
        [-2.6726e+00, -5.5529e+00, -1.2200e-01, -3.4113e+00, -4.7213e+00],
        [-3.5128e+00, -3.9415e+00, -4.4128e+00, -7.6311e+00, -6.3831e-02],
        [-3.0990e-03, -6.7913e+00, -1.0732e+01, -1.2591e+01, -6.2423e+00],
        [-8.7508e+00, -7.9279e-03, -8.5537e+00, -4.8941e+00, -9.8154e+00]])
Valores máximos e índices (tensor([-0.0036, -0.0114, -0.2819, -0.1220, -0.0638, -0.0031, -0.0079]), tensor([0, 1, 3, 2, 4, 0, 1]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}


In [ ]:
# Otra prueba
test_examples(test_data[2][0])
print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['el', 'agua', 'clara', 'fluye', 'por', 'el', 'rio']
La frase original preprocesada tensor([ 0, 67, 68, 46, 32,  0, 44])
Salida del modelo tensor([[-3.6377e-03, -6.2140e+00, -9.4082e+00, -1.1041e+01, -6.4813e+00],
        [-6.7706e+00, -1.2532e-02, -9.1952e+00, -4.5002e+00, -9.2278e+00],
        [-7.7423e+00, -3.7200e+00, -5.3346e+00, -3.0255e-02, -8.0744e+00],
        [-9.4140e+00, -8.6919e+00, -1.9223e-02, -4.2027e+00, -5.5637e+00],
        [-5.6938e+00, -7.2577e+00, -4.9830e+00, -9.4669e+00, -1.1064e-02],
        [-3.2324e-03, -7.3515e+00, -1.0614e+01, -1.2878e+01, -5.9684e+00],
        [-7.8867e+00, -3.5831e-03, -9.3864e+00, -5.7904e+00, -9.7164e+00]])
Valores máximos e índices (tensor([-0.0036, -0.0125, -0.0303, -0.0192, -0.0111, -0.0032, -0.0036]), tensor([0, 1, 3, 2, 4, 0, 1]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}


# Referencias 

[1] Guthrie, R. (2017). Tutorial. Sequence Models and Long-Short Term Memory Networks. Recuperado de https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

[2] LeCun,Y., Bengio, Y.,  & Hinton, G. (2015). Deep learning. Nature, 521(7553):436.

[3] Brownlee, J. (2017). What Are Word Embeddings for Text?. Recuperado de https://machinelearningmastery.com/what-are-word-embeddings/

[4] Bishop, C (2006). Pattern Recognition and Machine Learning. Springer.